# Inference

Down the road, you will need to make real-life predictions using the models that you've trained.

Inference is a breeze with AIQC because it persists all of the information that we need to preprocess our new samples and reconstruct our model.

Normally, the challenge with inference is being able to preprocess your new samples the same way as your processed your training samples. Additionally, if you provide labels with your new data for the purpose of evaluation, then PyTorch requires you to reconstruct parts of your model like your optimizer in order to calculate loss.

---

In [2]:
import aiqc
from aiqc import datum
from aiqc import tests

Below we're just making a trained model so that we have examples to work with for making inference-based predictions.

In [3]:
%%capture
queue_multiclass = tests.make_test_queue('keras_multiclass')
queue_multiclass.run_jobs()

## Predictor

Let's say that we have a trained model in the form of a `Predictor`,

In [4]:
predictor = queue_multiclass.jobs[0].predictors[0]

and that we have samples that we want to generate predictions for.

## New Splitset

In [5]:
df = datum.to_pandas('iris.tsv').sample(10)

In [6]:
df.tail()

,sepal_length,sepal_width,petal_length,petal_width,species
29,4.7,3.2,1.6,0.2,setosa
35,5.0,3.2,1.2,0.2,setosa
40,5.0,3.5,1.3,0.3,setosa
106,4.9,2.5,4.5,1.7,virginica
147,6.5,3.0,5.2,2.0,virginica


We'll fashion a new `Splitset` of the samples that we want to predict using the high-level API.

- Leave the `label_column` blank if you are conducting pure inference where you don't know the real Label/target. Otherwise, `splitset.label` will be used to generate metrics for your new predictions.
- Ultimately, any splits that you make will be ignored when calling `infer()` below as all samples from the `Dataset` will be utilized.

In [7]:
splitset = aiqc.Pipeline.Tabular.make(
    dataFrame_or_filePath = df
    , label_column = 'species'
)

## Run Inference

Then pass that `Splitset` to `Predictor.infer()`.

During `infer()`, it will validate that the schema of your new Splitset's `Feature` (the latest `Window` if provided) and `Label` (if provided) match the schema of the original training Splitset. 

- `Dataset.Tabular` schema includes column ordering and dtype.
- `Dataset.Image` schema includes Pillow size (height/width) and mode (color dimensions).

In [8]:
prediction = predictor.infer(splitset_id=splitset.id)

If you encoded your Labels or generated unsupervised encoded data, don't worry, the output will be `inverse_transform`'ed as seen below.

In [9]:
prediction.predictions

{'infer': array(['setosa', 'setosa', 'setosa', 'virginica', 'virginica',
        'virginica', 'versicolor', 'virginica', 'versicolor', 'virginica'],
       dtype=object)}

For more information on the `Prediction` object, reference the [Low-Level API](api_low_level.html) documentation.